In [1]:
import os

from huggingface_hub import notebook_login
notebook_login()

In [2]:
!pip uninstall -y bitsandbytes
!pip install --no-cache-dir bitsandbytes
!pip install --upgrade accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 293.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 302.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 278.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 267.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 213.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 265.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 301.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 246.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 224.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 325.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 318.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82

In [3]:
pip install np faiss-cpu sentence-transformers evaluate peft rouge bert_score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.1 MB/s eta 0:00:00
  Created wheel for np: filename=np-1.0.2-py3-none-any.whl size=13658 sha256=223f39c29b20e0fe785a44fcadc7825689e1d12d1937048fa5f2b3085d6a48f6
  Stored in directory: /root/.cache/pip/wheels/19/20/42/6ee214e617f78123903f603524d662ac6fa14154c3027fd992
Successfully built np


In [4]:
#!/usr/bin/env python
import argparse
import json
import time
import re
import os
import csv
import numpy as np
import torch
import faiss
import matplotlib.pyplot as plt
import evaluate

from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer, util
from peft import LoraConfig, get_peft_model
from rouge import Rouge
from tqdm import tqdm



# ========================================================
# 1. Mount Google Drive
# ========================================================

from google.colab import drive
drive.mount('/content/drive')


file_path = "/content/drive/My Drive/Dissertation/cuad_qa_dataset.json"


# --- Helper Functions ---


def preprocess_paragraph(paragraph):
    cleaned = re.sub(r"\n+", " ", paragraph)
    cleaned = re.sub(r"\s+", " ", cleaned).strip()
    return cleaned



def build_faiss_index(context, embedding_model, chunk_size=512, overlap=128):
    """
    Splits the context into fixed-length overlapping chunks,
    preprocesses them, and builds a FAISS index.
    """
    # Create overlapping chunks from the text
    chunks = []
    start = 0
    while start < len(context):
        chunk = context[start : start + chunk_size]
        chunks.append(chunk)
        start += (chunk_size - overlap)  # advance by chunk_size minus the overlap

    preprocessed_chunks = [preprocess_paragraph(chunk) for chunk in chunks if chunk.strip()]

    # Compute embeddings for all chunks
    chunk_embeddings = embedding_model.encode(preprocessed_chunks, convert_to_tensor=False)
    chunk_embeddings_np = np.array(chunk_embeddings).astype("float32")
    # Normalize embeddings for cosine similarity (or for L2 index)
    norms = np.linalg.norm(chunk_embeddings_np, axis=1, keepdims=True)
    chunk_embeddings_np = chunk_embeddings_np / norms
    embedding_dim = chunk_embeddings_np.shape[1]
    index = faiss.IndexFlatL2(embedding_dim)
    index.add(chunk_embeddings_np)
    print(f"FAISS Initialised: Indexed {index.ntotal} chunks.")

    preprocessed_paragraphs = preprocessed_chunks

    return index, preprocessed_paragraphs



# def build_faiss_index(context, embedding_model):
#     """Splits the context into paragraphs, preprocesses them, and builds a FAISS index."""
#     raw_paragraphs = context.split("\n\n")
#     preprocessed_paragraphs = [preprocess_paragraph(p) for p in raw_paragraphs if p.strip()]
#     # Compute embeddings for all paragraphs
#     paragraph_embeddings = embedding_model.encode(preprocessed_paragraphs, convert_to_tensor=False)
#     paragraph_embeddings_np = np.array(paragraph_embeddings).astype("float32")
#     # Normalize embeddings for cosine similarity (if using L2, normalized vectors can be compared)
#     norms = np.linalg.norm(paragraph_embeddings_np, axis=1, keepdims=True)
#     paragraph_embeddings_np = paragraph_embeddings_np / norms
#     embedding_dim = paragraph_embeddings_np.shape[1]
#     index = faiss.IndexFlatL2(embedding_dim)
#     index.add(paragraph_embeddings_np)
#     print(f"FAISS Initialised: Indexed {index.ntotal} paragraphs.")
#     return index, preprocessed_paragraphs

def retrieve_context(question, embedding_model, index, preprocessed_paragraphs, k=3, threshold=1):
    """Encodes the question and retrieves top k similar paragraphs. Uses a threshold to decide confidence."""
    question_embedding = embedding_model.encode([question]).astype("float32")
    question_embedding /= np.linalg.norm(question_embedding, axis=1, keepdims=True)
    distances, indices = index.search(question_embedding, k)
    best_distance = distances[0][0]
    if best_distance > threshold:
        print("\nLow confidence detected based on FAISS score.")
        return None  # Indicates low confidence

    selected_paragraphs = [preprocessed_paragraphs[i] for i in indices[0]]
    return "\n\n".join(selected_paragraphs)

def generate_answer(question, retrieved_context, tokenizer, model, device, gen_max_length=50):
    """Builds the final prompt and uses the generative model to produce an answer."""
    final_prompt = (
        "Answer the question in one or two sentences. "
        "Do not repeat the question or the context; only provide the final answer.\n"
        f"Question: {question}\n"
        f"Context: {retrieved_context}\n"
        "Final Answer:"
    )
    inputs = tokenizer(final_prompt, return_tensors="pt", truncation=True, max_length=1024)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    torch.cuda.empty_cache()
    generated_ids = model.generate(**inputs, max_new_tokens=gen_max_length)
    generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    if "Final Answer:" in generated_text:
        answer = generated_text.split("Final Answer:")[-1].strip()
    else:
        answer = generated_text.strip()
    return answer

def evaluate_generative_predictions(test_examples, tokenizer, model, sbert_model,bertscore_metric, rouge_metric, device, embedding_model):
    results_details = []
    semantic_scores = []
    bert_scores = []
    rouge_scores = []
    times = []

    for entry in tqdm(test_examples, desc="Evaluating test examples"):
        ex_id = entry["id"]
        question = entry["question"]
        contract_context = entry["context"]
        gold_text = entry["answers"]["text"][0].strip() if entry["answers"]["text"] else ""

        start_time = time.time()
        # Build FAISS index for the current contract context
        index, paragraphs = build_faiss_index(contract_context, embedding_model)

        # Retrieve context from the current contract context
        retrieved_context = retrieve_context(question, embedding_model, index, paragraphs)

        if retrieved_context is None:
            # If retrieval fails, we set the prediction as an empty string
            pred_text = ""
            duration = time.time() - start_time
            times.append(duration)
        else:
            # Build final prompt using the retrieved context
            final_prompt = (
                "Answer the question in one or two sentences. "
                "Do not repeat the question or the context; only provide the final answer.\n"
                f"Question: {question}\n"
                f"Context: {retrieved_context}\n"
                "Final Answer:"
            )

            inputs = tokenizer(final_prompt, return_tensors="pt", max_length=1024, truncation=True)
            inputs = {k: v.to(device) for k, v in inputs.items()}
            generated_ids = model.generate(**inputs, max_length=1024)
            duration = time.time() - start_time
            times.append(duration)
            pred_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True).strip()
            # Extract only the final answer part
            if "Final Answer:" in pred_text:
              pred_text = pred_text.split("Final Answer:")[-1].strip()



        # Evaluate metrics with proper empty-answer handling.
        if gold_text == "" and pred_text == "":
            sem_sim = 1.0
            b_score = 1.0
            r_score = 1.0
        elif gold_text == "" or pred_text == "":
            sem_sim = 0.0
            b_score = 0.0
            r_score = 0.0
        else:
            # Compute semantic similarity using cosine similarity of SBERT embeddings.
            emb_gold = sbert_model.encode(gold_text, convert_to_tensor=True)
            emb_pred = sbert_model.encode(pred_text, convert_to_tensor=True)
            sem_sim = float(util.pytorch_cos_sim(emb_gold, emb_pred).item())

            # Compute BERTScore using the evaluate library.
            b_result = bertscore_metric.compute(predictions=[pred_text], references=[gold_text], lang="en")
            b_score = b_result["f1"][0]

            # Compute ROUGE-L score.
            r = rouge_metric.get_scores(pred_text, gold_text)
            r_score = r[0]["rouge-l"]["f"]

        semantic_scores.append(sem_sim)
        bert_scores.append(b_score)
        rouge_scores.append(r_score)

        results_details.append({
            "id": ex_id,
            "question": question,
            "gold_answer": gold_text,
            "model_answer": pred_text,
            "semantic_similarity": sem_sim,
            "bert_score": b_score,
            "rouge_l_f1": r_score,
            "eval_time_sec": duration
        })

    overall_results = {
        "avg_semantic_similarity": float(np.mean(semantic_scores)),
        "avg_bert_score": float(np.mean(bert_scores)),
        "avg_rouge_l_f1": float(np.mean(rouge_scores)),
        "avg_eval_time_sec": float(np.mean(times))
    }

    return overall_results, results_details

def save_results_to_csv(details, filename):
    if details:
        keys = details[0].keys()
        with open(filename, "w", newline="") as f:
            dict_writer = csv.DictWriter(f, keys)
            dict_writer.writeheader()
            dict_writer.writerows(details)
        print(f"Saved evaluation details to {filename}")
    else:
        print("No details to save.")


def plot_model_comparison(aggregated_results, save_dir="./"):
    models = [r["model"] for r in aggregated_results]
    avg_rouge = [r["avg_rouge_l_f1"] for r in aggregated_results]
    avg_semantic = [r["avg_semantic_similarity"] for r in aggregated_results]
    avg_bert = [r["avg_bert_score"] for r in aggregated_results]
    avg_time = [r.get("avg_eval_time_sec", 0) for r in aggregated_results]

    x = np.arange(len(models))
    width = 0.25  # Width for each bar

    # Plot metrics comparison
    plt.figure(figsize=(10, 6))
    plt.bar(x - width, avg_rouge, width, label="ROUGE-L")
    plt.bar(x, avg_semantic, width, label="Semantic Similarity")
    plt.bar(x + width, avg_bert, width, label="BERTScore")
    plt.xlabel("Model")
    plt.ylabel("Metric Score")
    plt.title("Evaluation Metrics Comparison")
    plt.xticks(x, models, rotation=45)
    plt.legend()
    plt.tight_layout()
    # Save the figure to a file
    model_comparison_path = os.path.join(save_dir, "model_comparison.png")
    plt.savefig(model_comparison_path)
    plt.show()
    print(f"Saved model comparison plot to {model_comparison_path}")

    # Plot evaluation time per model
    plt.figure(figsize=(10, 6))
    plt.bar(x, avg_time, width=0.5, color="skyblue")
    plt.xlabel("Model")
    plt.ylabel("Average Evaluation Time (sec)")
    plt.title("Evaluation Time per Model")
    plt.xticks(x, models, rotation=45)
    plt.tight_layout()
    time_plot_path = os.path.join(save_dir, "evaluation_time.png")
    plt.savefig(time_plot_path)
    plt.show()
    print(f"Saved evaluation time plot to {time_plot_path}")

def plot_line_metrics(details, save_dir="./"):
    # Extract metric values for each example
    indices = list(range(len(details)))
    semantic_scores = [d["semantic_similarity"] for d in details]
    bert_scores = [d["bert_score"] for d in details]
    rouge_scores = [d["rouge_l_f1"] for d in details]

    plt.figure(figsize=(12, 6))
    plt.plot(indices, semantic_scores, label="Semantic Similarity", marker='o', linestyle='-')
    plt.plot(indices, bert_scores, label="BERTScore", marker='o', linestyle='-')
    plt.plot(indices, rouge_scores, label="ROUGE-L", marker='o', linestyle='-')

    plt.xlabel("Example Index")
    plt.ylabel("Score")
    plt.title("Evaluation Metrics per Example")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()

    line_plot_path = os.path.join(save_dir, "line_metrics.png")
    plt.savefig(line_plot_path)
    plt.show()


# --- Main Pipeline ---

def main():
    # Set up device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # generative model and tokenizer for Falcon-7B
    model_name = "tiiuae/falcon-7b"  # Updated model name
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16,
        device_map="auto"
    )
    print("✅ Falcon-7B Loaded")



    # Initialize embedding model for retrieval (SBERT)
    embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

    # Initialize ROUGE metric for evaluation
    rouge_metric = Rouge()

    # Load the BERTScore metric
    bertscore_metric = evaluate.load("bertscore")

    # Load your dataset (assuming JSON format with a "test" split)
    with open(file_path, "r", encoding="utf-8") as f:
        raw_data = json.load(f)
    # For demonstration, we use a small subset
    test_examples = raw_data["train"][:500]
    print(f"Loaded {len(test_examples)} test examples.")


    # Evaluate generative predictions over the test examples
    overall_results, details = evaluate_generative_predictions(
        test_examples, tokenizer, model, embedding_model, bertscore_metric, rouge_metric, device, embedding_model
    )

    print("\nOverall Evaluation Results:")
    print(overall_results)

    # Save overall evaluation results to a JSON file
    with open("./overall_evaluation_results.json", "w", encoding="utf-8") as f:
        json.dump(overall_results, f, indent=4)
    print("Saved overall evaluation results to overall_evaluation_results.json")

    # Save detailed per-example results to CSV
    save_results_to_csv(details, "./detailed_evaluation_results.csv")
    # For comparison, aggregate results from different models


    aggregated_results = []
    overall_results["model"] = "falcon-7b"
    aggregated_results.append(overall_results)
    # Plot and save bar charts for overall metrics
    plot_model_comparison(aggregated_results, save_dir="./")

    # Plot and save the line graph for per-example metrics
    plot_line_metrics(details, save_dir="./")

if __name__ == "__main__":
    main()

Mounted at /content/drive


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

...

FAISS Initialised: Indexed 142 chunks.


Evaluating test examples:   1%|          | 5/500 [04:13<7:46:40, 56.57s/it]Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


FAISS Initialised: Indexed 142 chunks.


Evaluating test examples:   1%|          | 5/500 [04:28<7:22:49, 53.68s/it]


KeyboardInterrupt: 